# Generativa nätverk

Recurrent Neural Networks (RNNs) och deras varianter med grindade celler, såsom Long Short Term Memory Cells (LSTMs) och Gated Recurrent Units (GRUs), gav en mekanism för språkmodellering, det vill säga de kan lära sig ordordning och ge förutsägelser för nästa ord i en sekvens. Detta gör det möjligt att använda RNNs för **generativa uppgifter**, såsom vanlig textgenerering, maskinöversättning och till och med bildbeskrivning.

I RNN-arkitekturen som vi diskuterade i föregående enhet, producerade varje RNN-enhet nästa dolda tillstånd som en utgång. Men vi kan också lägga till en annan utgång till varje återkommande enhet, vilket skulle göra det möjligt för oss att generera en **sekvens** (som är lika lång som den ursprungliga sekvensen). Dessutom kan vi använda RNN-enheter som inte tar emot en inmatning vid varje steg, utan bara tar en initial tillståndsvektor och sedan producerar en sekvens av utgångar.

I denna notebook kommer vi att fokusera på enkla generativa modeller som hjälper oss att generera text. För enkelhetens skull ska vi bygga ett **teckennivånätverk**, som genererar text bokstav för bokstav. Under träningen behöver vi ta en textkorpus och dela upp den i teckensekvenser.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

## Bygga teckenordförråd

För att bygga ett generativt nätverk på teckennivå behöver vi dela upp texten i enskilda tecken istället för ord. `TextVectorization`-lagret som vi har använt tidigare kan inte göra detta, så vi har två alternativ:

* Ladda text manuellt och göra tokenisering "för hand", som i [detta officiella Keras-exempel](https://keras.io/examples/generative/lstm_character_level_text_generation/)
* Använda `Tokenizer`-klassen för tokenisering på teckennivå.

Vi kommer att välja det andra alternativet. `Tokenizer` kan också användas för att tokenisera till ord, så det bör vara enkelt att växla mellan tokenisering på teckennivå och ordnivå.

För att göra tokenisering på teckennivå behöver vi ange parametern `char_level=True`:


In [2]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True,lower=False)
tokenizer.fit_on_texts([x['title'].numpy().decode('utf-8') for x in ds_train])

Vi vill också använda en speciell token för att beteckna **slut på sekvens**, som vi kommer att kalla `<eos>`. Låt oss lägga till den manuellt i vokabulären:


In [3]:
eos_token = len(tokenizer.word_index)+1
tokenizer.word_index['<eos>'] = eos_token

vocab_size = eos_token + 1

Nu, för att koda text till sekvenser av siffror, kan vi använda:


In [4]:
tokenizer.texts_to_sequences(['Hello, world!'])

[[48, 2, 10, 10, 5, 44, 1, 25, 5, 8, 10, 13, 78]]

## Träna ett generativt RNN för att skapa titlar

Så här kommer vi att träna ett RNN för att generera nyhetstitlar. Vid varje steg tar vi en titel, som matas in i ett RNN, och för varje inmatad tecken ber vi nätverket att generera nästa utmatade tecken:

![Bild som visar ett exempel på RNN-generering av ordet 'HELLO'.](../../../../../translated_images/sv/rnn-generate.56c54afb52f9781d.webp)

För det sista tecknet i vår sekvens kommer vi att be nätverket att generera `<eos>`-token.

Den största skillnaden med det generativa RNN som vi använder här är att vi kommer att ta en utmatning från varje steg i RNN, och inte bara från den sista cellen. Detta kan uppnås genom att specificera parametern `return_sequences` till RNN-cellen.

Således, under träningen, skulle en inmatning till nätverket vara en sekvens av kodade tecken av en viss längd, och en utmatning skulle vara en sekvens av samma längd, men förskjuten med ett element och avslutad med `<eos>`. En minibatch kommer att bestå av flera sådana sekvenser, och vi behöver använda **padding** för att justera alla sekvenser.

Låt oss skapa funktioner som kommer att transformera datasetet åt oss. Eftersom vi vill fylla ut sekvenser på minibatch-nivå, kommer vi först att batcha datasetet genom att anropa `.batch()`, och sedan `map` för att utföra transformationen. Så, transformationsfunktionen kommer att ta en hel minibatch som parameter:


In [5]:
def title_batch(x):
    x = [t.numpy().decode('utf-8') for t in x]
    z = tokenizer.texts_to_sequences(x)
    z = tf.keras.preprocessing.sequence.pad_sequences(z)
    return tf.one_hot(z,vocab_size), tf.one_hot(tf.concat([z[:,1:],tf.constant(eos_token,shape=(len(z),1))],axis=1),vocab_size)

Några viktiga saker som vi gör här:
* Vi börjar med att extrahera den faktiska texten från sträng-tensorn
* `text_to_sequences` omvandlar listan av strängar till en lista av heltals-tensorer
* `pad_sequences` fyller ut dessa tensorer till deras maximala längd
* Slutligen one-hot-kodar vi alla tecken, och gör även förskjutningen och lägger till `<eos>`. Vi kommer snart att se varför vi behöver one-hot-kodade tecken

Den här funktionen är dock **Pythonisk**, dvs. den kan inte automatiskt översättas till Tensorflow:s beräkningsgraf. Vi kommer att få fel om vi försöker använda den här funktionen direkt i `Dataset.map`-funktionen. Vi behöver kapsla in detta Pythoniska anrop genom att använda `py_function`-omslutaren:


In [6]:
def title_batch_fn(x):
    x = x['title']
    a,b = tf.py_function(title_batch,inp=[x],Tout=(tf.float32,tf.float32))
    return a,b

> **Observera**: Att skilja mellan Pythoniska och Tensorflow-transformationsfunktioner kan verka lite för komplext, och du kanske undrar varför vi inte transformerar datasetet med standardfunktioner i Python innan vi skickar det till `fit`. Även om detta definitivt är möjligt, har användningen av `Dataset.map` en stor fördel, eftersom datatransformationspipelinan körs med Tensorflows beräkningsgraf. Detta drar nytta av GPU-beräkningar och minimerar behovet av att överföra data mellan CPU och GPU.

Nu kan vi bygga vårt generatornätverk och börja träna. Det kan baseras på vilken återkommande cell som helst som vi diskuterade i föregående enhet (enkel, LSTM eller GRU). I vårt exempel kommer vi att använda LSTM.

Eftersom nätverket tar tecken som indata och vokabulärstorleken är ganska liten, behöver vi inget inbäddningslager; one-hot-kodad indata kan direkt skickas in i LSTM-cellen. Utgångslagret skulle vara en `Dense`-klassificerare som omvandlar LSTM-utgången till one-hot-kodade tokennummer.

Dessutom, eftersom vi arbetar med sekvenser av varierande längd, kan vi använda ett `Masking`-lager för att skapa en mask som ignorerar den utfyllda delen av strängen. Detta är inte strikt nödvändigt, eftersom vi inte är särskilt intresserade av allt som går bortom `<eos>`-token, men vi kommer att använda det för att få lite erfarenhet av denna typ av lager. `input_shape` skulle vara `(None, vocab_size)`, där `None` indikerar sekvenser av varierande längd, och utgångsformen är också `(None, vocab_size)`, som du kan se från `summary`:


In [7]:
model = keras.models.Sequential([
    keras.layers.Masking(input_shape=(None,vocab_size)),
    keras.layers.LSTM(128,return_sequences=True),
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy')

model.fit(ds_train.batch(8).map(title_batch_fn))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, None, 84)          0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         109056    
_________________________________________________________________
dense (Dense)                (None, None, 84)          10836     
Total params: 119,892
Trainable params: 119,892
Non-trainable params: 0
_________________________________________________________________
15000/15000 [==============================] - 229s 15ms/step - loss: 1.5385


## Generera output

Nu när vi har tränat modellen vill vi använda den för att generera output. Först och främst behöver vi ett sätt att avkoda text som representeras av en sekvens av tokennummer. För att göra detta kan vi använda funktionen `tokenizer.sequences_to_texts`; dock fungerar den inte särskilt bra med tokenisering på teckennivå. Därför kommer vi att ta en ordbok med tokens från tokenizer (kallad `word_index`), bygga en omvänd karta och skriva vår egen avkodningsfunktion:


In [10]:
reverse_map = {val:key for key, val in tokenizer.word_index.items()}

def decode(x):
    return ''.join([reverse_map[t] for t in x])

Nu börjar vi med en sträng `start`, kodar den till en sekvens `inp`, och sedan vid varje steg anropar vi vårt nätverk för att förutsäga nästa tecken.

Utdata från nätverket `out` är en vektor med `vocab_size` element som representerar sannolikheten för varje token, och vi kan hitta det mest sannolika token-numret genom att använda `argmax`. Därefter lägger vi till detta tecken i den genererade listan av tokens och fortsätter med genereringen. Denna process att generera ett tecken upprepas `size` gånger för att generera önskat antal tecken, och vi avslutar tidigare om `eos_token` påträffas.


In [12]:
def generate(model,size=100,start='Today '):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            nc = tf.argmax(out)
            if nc==eos_token:
                break
            chars.append(nc.numpy())
            inp = inp+[nc]
        return decode(chars)
    
generate(model)

'Today #39;s lead to strike for the strike for the strike for the strike (AFP)'

## Sampla output under träning

Eftersom vi inte har några användbara mått som *noggrannhet*, är det enda sättet att se att vår modell förbättras genom att **sampla** genererade strängar under träning. För att göra detta kommer vi att använda **callbacks**, dvs. funktioner som vi kan skicka till `fit`-funktionen, och som kommer att anropas periodiskt under träningen.


In [13]:
sampling_callback = keras.callbacks.LambdaCallback(
  on_epoch_end = lambda batch, logs: print(generate(model))
)

model.fit(ds_train.batch(8).map(title_batch_fn),callbacks=[sampling_callback],epochs=3)

Epoch 1/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.2703
Today #39;s a lead in the company for the strike
Epoch 2/3
15000/15000 [==============================] - 227s 15ms/step - loss: 1.2057
Today #39;s the Market Service on Security Start (AP)
Epoch 3/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.1752
Today #39;s a line on the strike to start for the start


Det här exemplet genererar redan ganska bra text, men det kan förbättras på flera sätt:

* **Mer text**. Vi har bara använt titlar för vår uppgift, men du kanske vill experimentera med fullständig text. Kom ihåg att RNN:er inte är särskilt bra på att hantera långa sekvenser, så det är vettigt att antingen dela upp dem i kortare meningar eller alltid träna på en fast sekvenslängd av ett fördefinierat värde `num_chars` (till exempel 256). Du kan försöka ändra exemplet ovan till en sådan arkitektur, med hjälp av [officiell Keras-handledning](https://keras.io/examples/generative/lstm_character_level_text_generation/) som inspiration.

* **Flerskikts-LSTM**. Det kan vara värt att prova 2 eller 3 lager av LSTM-celler. Som vi nämnde i den tidigare enheten, extraherar varje lager av LSTM vissa mönster från texten, och i fallet med en generator på teckennivå kan vi förvänta oss att det lägre LSTM-lagret ansvarar för att extrahera stavelser, och de högre lagren - för ord och ordkombinationer. Detta kan enkelt implementeras genom att skicka ett parameter för antal lager till LSTM-konstruktorn.

* Du kanske också vill experimentera med **GRU-enheter** och se vilka som presterar bättre, samt med **olika storlekar på dolda lager**. Ett för stort dolt lager kan leda till överanpassning (t.ex. att nätverket lär sig exakt text), medan en mindre storlek kanske inte ger ett bra resultat.


## Mjuk textgenerering och temperatur

I den tidigare definitionen av `generate` valde vi alltid tecknet med högst sannolikhet som nästa tecken i den genererade texten. Detta resulterade ofta i att texten "cirkulerade" mellan samma teckensekvenser om och om igen, som i detta exempel:
```
today of the second the company and a second the company ...
```

Men om vi tittar på sannolikhetsfördelningen för nästa tecken, kan det vara så att skillnaden mellan de högsta sannolikheterna inte är särskilt stor, t.ex. ett tecken kan ha sannolikheten 0,2, ett annat 0,19, etc. Till exempel, när vi letar efter nästa tecken i sekvensen '*play*', kan nästa tecken lika gärna vara ett mellanslag eller **e** (som i ordet *player*).

Detta leder oss till slutsatsen att det inte alltid är "rättvist" att välja tecknet med högst sannolikhet, eftersom att välja det näst högsta fortfarande kan leda till meningsfull text. Det är klokare att **sampla** tecken från sannolikhetsfördelningen som ges av nätverkets output.

Denna sampling kan göras med hjälp av funktionen `np.multinomial`, som implementerar den så kallade **multinomialfördelningen**. En funktion som implementerar denna **mjuka** textgenerering definieras nedan:


In [33]:
def generate_soft(model,size=100,start='Today ',temperature=1.0):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            probs = tf.exp(tf.math.log(out)/temperature).numpy().astype(np.float64)
            probs = probs/np.sum(probs)
            nc = np.argmax(np.random.multinomial(1,probs,1))
            if nc==eos_token:
                break
            chars.append(nc)
            inp = inp+[nc]
        return decode(chars)

words = ['Today ','On Sunday ','Moscow, ','President ','Little red riding hood ']
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"\n--- Temperature = {i}")
    for j in range(5):
        print(generate_soft(model,size=300,start=words[j],temperature=i))


--- Temperature = 0.3
Today #39;s strike #39; to start at the store return
On Sunday PO to Be Data Profit Up (Reuters)
Moscow, SP wins straight to the Microsoft #39;s control of the space start
President olding of the blast start for the strike to pay &lt;b&gt;...&lt;/b&gt;
Little red riding hood ficed to the spam countered in European &lt;b&gt;...&lt;/b&gt;

--- Temperature = 0.8
Today countie strikes ryder missile faces food market blut
On Sunday collores lose-toppy of sale of Bullment in &lt;b&gt;...&lt;/b&gt;
Moscow, IBM Diffeiting in Afghan Software Hotels (Reuters)
President Ol Luster for Profit Peaced Raised (AP)
Little red riding hood dace on depart talks #39; bank up

--- Temperature = 1.0
Today wits House buiting debate fixes #39; supervice stake again
On Sunday arling digital poaching In for level
Moscow, DS Up 7, Top Proble Protest Caprey Mamarian Strike
President teps help of roubler stepted lessabul-Dhalitics (AFP)
Little red riding hood signs on cash in Carter-youb

---

KeyError: 0

Vi har introducerat en ytterligare parameter kallad **temperatur**, som används för att indikera hur strikt vi ska hålla oss till den högsta sannolikheten. Om temperaturen är 1,0 gör vi rättvis multinomial sampling, och när temperaturen går mot oändligheten - blir alla sannolikheter lika, och vi väljer nästa tecken slumpmässigt. I exemplet nedan kan vi observera att texten blir meningslös när vi ökar temperaturen för mycket, och den liknar "cyklisk" hårdgenererad text när den närmar sig 0.



---

**Ansvarsfriskrivning**:  
Detta dokument har översatts med hjälp av AI-översättningstjänsten [Co-op Translator](https://github.com/Azure/co-op-translator). Även om vi strävar efter noggrannhet, bör du vara medveten om att automatiserade översättningar kan innehålla fel eller felaktigheter. Det ursprungliga dokumentet på dess ursprungliga språk bör betraktas som den auktoritativa källan. För kritisk information rekommenderas professionell mänsklig översättning. Vi ansvarar inte för eventuella missförstånd eller feltolkningar som uppstår vid användning av denna översättning.
